# Optimize and Evaluate Image Generation Models

- **Goal:** Show a standard workflow for optimizing and evaluating a image generation model.
- **Model:** [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)
- **Dataset:** [data-is-better-together/open-image-preferences-v1](https://huggingface.co/datasets/data-is-better-together/open-image-preferences-v1)
- **Libraries:** [diffusers](https://github.com/huggingface/diffusers)
- **Optimization Algorithms:** quantiser(hqq_diffusers), compiler(torch_compile)
- **Evaluation Metrics:**
    - **Base Metrics:** elapsed_time
    - **Stateful Metrics:** lpi, ssim, psnr, clip_score, cmmd

## Getting Started


### Install the dependencies

To install the dependencies, run the following command:

In [ ]:
!pip install pruna

### Set the device

Normally, we would set the device to the best available device to make the most out of the optimization process.

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"